In [2]:
import ee
import geemap

# Initialize Google Earth Engine


In [3]:
# ee.Authenticate()
ee.Initialize()

In [7]:
# Define the region of interest (geojson file for farm plots)
roi = geemap.geojson_to_ee("/Users/joy/Documents/code/gMapExperiements/data/ecosia-single.geojson")

In [ ]:
# map = geemap.Map()
# map.addLayer(roi, {}, 'Farm Plots')
# map.centerObject(roi, zoom=15)
# map


In [15]:
# Define the date range for image collection
start_date = '2022-01-01'
end_date = '2022-12-31'

# Function to mask clouds and shadows
def maskClouds(image):
    # Bits 10 and 11 represent clouds and cirrus, respectively
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Get the pixel QA band
    qa = image.select('QA60')

    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0) and (qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    # Return the masked image, scaled to NDVI range
    return image.updateMask(mask).divide(10000)

# Function to compute NDVI
def computeNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Sentinel-2 image collection
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterBounds(roi)
                #  .filterDate(start_date, end_date)
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                 .map(maskClouds)
                 .map(computeNDVI))



In [33]:
months=ee.List.sequence(1,12);
years=ee.List.sequence(2018,2022);
# var pentads=ee.List.sequence(1,6)

def calculateMonthlyRainfall(year):
    def calculateMonthlyPentads(month):
            return (s2_collection
                    .filter(ee.Filter.calendarRange(year, year, "year"))
                    .filter(ee.Filter.calendarRange(month, month, "month"))
                    .reduce(ee.Reducer.mean())
                    .set({"month": month, "year": year})
                   )
    return months.map(calculateMonthlyPentads)

# Flatten the result
monthlyRainfall = years.map(calculateMonthlyRainfall).flatten()
monthlyRainfall

In [32]:
monthlyRainfall.getInfo()

EEException: String.cat, argument 'string2': Invalid type.
Expected type: String.
Actual type: Float.
Actual value: 2018.0

In [ ]:
# Resample the image collection to monthly frequency
monthly_collection = s2_collection.filter(ee.Filter.calendarRange(1, 12, 'month'))

# Compute the long-term average NDVI for each month
long_term_average = monthly_collection.mean()

# Compute NDVI anomalies for each month
def computeAnomaly(image):
    # Get the month from the image timestamp
    month = image.date().get('month')

    # Compute the anomaly by subtracting the long-term average from the image
    anomaly = image.select('NDVI').subtract(long_term_average.select('NDVI')).rename('NDVI_Anomaly')

    # Return the image with the anomaly band and month property
    return image.addBands(anomaly).set('month', month)

anomaly_collection = monthly_collection.map(computeAnomaly)

# Filter images for months where NDVI couldn't be computed due to cloud cover
filtered_collection = anomaly_collection.filter(ee.Filter.notNull(['NDVI_Anomaly']))

# Print the image collection
print('Filtered Image Collection:', filtered_collection)

# Visualization
map = geemap.Map()
map.centerObject(roi, zoom=15)
vis_params = {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'green']}
map.addLayer(long_term_average.select('NDVI'), vis_params, 'Long-term Average NDVI')
map.addLayer(filtered_collection.select('NDVI_Anomaly'), vis_params, 'NDVI Anomaly')
map.addLayer(roi, {}, 'Farm Plots')
map.addLayerControl()
map
